In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from roler.distributions import *
from roler.simulation import Simulator
from roler.model import ModelPrior

In [79]:
prior = ModelPrior(
    individuals_local=IntDistribution(100, 10000),
    add_individuals_meta=IntDistribution(100, 10000),
    species_meta=IntDistribution(10, 500),
    speciation_local=FloatDistribution(0.0, 1e-4),
    mutation_rate=FloatDistribution(1e-7, 1e-5),
)

In [82]:
from dataclasses import asdict
import json

json_data = json.dumps(asdict(prior), indent=4)
print(json_data)

{
    "individuals_local": {
        "low": 99.5,
        "high": 10000.5
    },
    "add_individuals_meta": {
        "low": 99.5,
        "high": 10000.5
    },
    "species_meta": {
        "low": 9.5,
        "high": 500.5
    },
    "speciation_local": {
        "low": 0.0,
        "high": 0.0001
    },
    "speciation_meta": 1.0,
    "extinction_meta": 0.8,
    "env_sigma": 0.0,
    "trait_sigma": 1.0,
    "comp_sigma": 0,
    "dispersal_prob": 0.01,
    "mutation_rate": {
        "low": 1e-07,
        "high": 1e-05
    },
    "equilib_escape": 0.0,
    "num_basepairs": 500,
    "alpha": 1.0,
    "neut_delta": 1.0,
    "env_comp_delta": 0.5,
    "init_type": "oceanic_island",
    "niter": 500000,
    "niterTimestep": 500000
}


In [40]:
tensor_sample = prior.sample()
print(tensor_sample)
params = prior.get_params(tensor_sample)
print("Simulation Params")
print(params.model_dump_json(indent=4))

tensor([4.5303e+02, 1.7122e+03, 2.3474e+01, 7.0716e-05, 5.5561e-06])
Simulation Params
{
    "individuals_local": 453,
    "individuals_meta": 2165,
    "species_meta": 23,
    "speciation_local": 0.00007071551226545125,
    "speciation_meta": 1.0,
    "extinction_meta": 0.8,
    "env_sigma": 0.0,
    "trait_sigma": 1.0,
    "comp_sigma": 0.0,
    "dispersal_prob": 0.01,
    "mutation_rate": 5.556126779993065e-6,
    "equilib_escape": 0.0,
    "num_basepairs": 500,
    "alpha": 1.0,
    "neut_delta": 1.0,
    "env_comp_delta": 0.5,
    "init_type": "oceanic_island",
    "niter": 500000,
    "niterTimestep": 500000
}


In [96]:
simulator = Simulator(prior=prior)
simulator(prior.sample()) # sample from the provided prior

,richness,hill_abund_1,hill_abund_2,hill_abund_3,hill_abund_4,hill_trait_1,hill_trait_2,hill_trait_3,hill_trait_4
0,95,14.299959,4.036702,2.910407,2.585989,22.089784,13.758066,11.811513,10.868421


In [19]:
from roler.datasets import DatasetGenerator

generator = DatasetGenerator(simulator)
dataset = generator.generate_dataset(10000, 14)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   8 tasks      | elapsed:    0.5s
/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=14)]: Done  22 tasks      | elapsed:    1.5s
[Parallel(n_jobs=14)]: Done  51 tasks      | elapsed:    2.2s
/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.12/site-packages/joblib/externals/loky/process_

In [65]:
params = dataset["params"]
output = dataset["output"]

In [66]:
print(params.shape)
print(output.shape)

(9908, 5)
(9908, 9)


In [23]:
params.to_csv("data/params.csv")
output.to_csv("data/output.csv")

In [67]:
import torch
x = torch.tensor(params.to_numpy(), dtype=torch.float32).to("mps")
y = torch.tensor(output.to_numpy(), dtype=torch.float32).to("mps")

mask = torch.isfinite(x).all(dim=1) & torch.isfinite(y).all(dim=1)
x = x[mask]
y = y[mask]

In [68]:
print(x.shape, y.shape)

torch.Size([9907, 5]) torch.Size([9907, 9])


In [69]:
from sbi.inference import SNPE

# gpu_prior = prior.get_uniform("mps")

snpe = SNPE(prior=prior)
density_estimator = snpe.append_simulations(x, y).train()
posterior = snpe.build_posterior(density_estimator)

/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:157: UserWarning: Data x has device 'mps:0'. Moving x to the data_device 'cpu'. Training will proceed on device 'cpu'.
  theta, x = validate_theta_and_x(
/Users/micahtilton/Documents/GitHub/likelihood-free-inference/.venv/lib/python3.12/site-packages/sbi/inference/trainers/npe/npe_base.py:157: UserWarning: Parameters theta has device 'mps:0'. Moving theta to the data_device 'cpu'. Training will proceed on device 'cpu'.
  theta, x = validate_theta_and_x(


 Neural network successfully converged after 244 epochs.

In [85]:
# Save the posterior to a file
torch.save(posterior, "posterior.pt")

# Load the posterior back into memory
loaded_posterior = torch.load("posterior.pt", weights_only=False)

# Test if the loaded posterior is the same as the original
print("Posterior loaded successfully:", isinstance(loaded_posterior, type(posterior)))

Posterior loaded successfully: True


In [102]:
generator = DatasetGenerator(simulator)
true_data = generator.generate_dataset(samples=1)
true_data

true_data = generator.generate_dataset(samples=1)
theta_true = true_data["params"].to_numpy()[-1]
x_obs = true_data["output"].to_numpy()[-1]
print(theta_true, x_obs)

[8.1111455e+03 9.2288955e+03 1.5240854e+02 4.5958292e-05 2.1403209e-06] [61.         10.21070189  3.78987272  2.81185383  2.51014179 19.60053646
 11.41592446  9.50776885  8.70705185]


In [101]:
true_data = generator.generate_dataset(samples=1)
theta_true = true_data["params"].to_numpy()[-1]
x_obs = true_data["output"].to_numpy()[-1]

theta_true = torch.tensor(theta_true, dtype=torch.float32)
x_obs = torch.tensor(x_obs, dtype=torch.float32)

print("Observed simulation output:", x_obs)

# posterior_cpu = snpe.build_posterior(density_estimator, sample_with="")
# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = loaded_posterior.sample((10000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)


print(prior.get_params(theta_true))
print(prior.get_params(posterior_mean))

Observed simulation output: tensor([83.0000, 10.6241,  3.4432,  2.5880,  2.3305, 19.1789, 11.7092,  9.7711,
         8.7843])


Drawing 10000 posterior samples: 10337it [00:00, 153975.85it/s]           

Posterior samples shape: torch.Size([10000, 5])
Posterior mean estimate: tensor([7.6766e+03, 4.8802e+03, 2.1878e+02, 5.1047e-05, 5.3437e-06])

Theta True      : tensor([7.8425e+03, 9.7359e+03, 2.5300e+02, 9.4709e-05, 1.8822e-06])
Theta Prediction: tensor([7.6766e+03, 4.8802e+03, 2.1878e+02, 5.1047e-05, 5.3437e-06])
individuals_local=7843 individuals_meta=17579 species_meta=253 speciation_local=9.470901568420231e-05 speciation_meta=1.0 extinction_meta=0.8 env_sigma=0.0 trait_sigma=1.0 comp_sigma=0.0 dispersal_prob=0.01 mutation_rate=1.8821747289621271e-06 equilib_escape=0.0 num_basepairs=500 alpha=1.0 neut_delta=1.0 env_comp_delta=0.5 init_type='oceanic_island' niter=500000 niterTimestep=500000
individuals_local=7677 individuals_meta=12557 species_meta=219 speciation_local=5.104676529299468e-05 speciation_meta=1.0 extinction_meta=0.8 env_sigma=0.0 trait_sigma=1.0 comp_sigma=0.0 dispersal_prob=0.01 mutation_rate=5.343692464521155e-06 equilib_escape=0.0 num_basepairs=500 alpha=1.0 neut_de